In [1]:
import os
import tensorflow as tf

In [2]:
file_name = "index_HanDB_Train.tfrecord"
record_iterator = tf.python_io.tf_record_iterator(path=file_name)

numb_of_data=0
for record in record_iterator:
    numb_of_data+=1

tf_record_filename_queue = tf.train.string_input_producer(tf.train.match_filenames_once("./"+file_name))
#f=open("./test_record.tfrecord",'rb')

In [3]:
def read_and_decode(filename_queue, batch_size):
    tf_record_reader=tf.TFRecordReader()
    _, tf_record_serialized = tf_record_reader.read(tf_record_filename_queue)

    tf_record_features = tf.parse_single_example(
        tf_record_serialized,
        features={
            'label': tf.FixedLenFeature([], tf.int64),
            'width': tf.FixedLenFeature([], tf.int64),
            'height': tf.FixedLenFeature([], tf.int64),
            'image': tf.FixedLenFeature([], tf.string),
        })
    tf_record_image = tf.decode_raw(tf_record_features['image'],tf.uint8)
    
    tf_record_image = tf.cast(tf_record_image, tf.float32)

    tf_record_image = tf.reshape(tf_record_image, [64,64,1])

    tf_record_label = tf.cast(tf_record_features['label'],tf.int64)
    
    images, labels = tf.train.shuffle_batch([tf_record_image, tf_record_label],
                                         batch_size = batch_size,
                                         capacity=20*batch_size,
                                         min_after_dequeue=5*batch_size )
    return images, labels

In [4]:
input_images = tf.placeholder(tf.float32,[None,64,64,1])
input_labels = tf.placeholder(tf.int64,[None])
keep_prob = tf.placeholder(tf.float32)
print(input_images)
print(input_labels)

Tensor("Placeholder:0", shape=(?, 64, 64, 1), dtype=float32)
Tensor("Placeholder_1:0", shape=(?,), dtype=int64)


In [5]:
W1 = tf.Variable(tf.random_normal([5,5,1,32], stddev=0.01))
L1 = tf.nn.conv2d(input_images,W1, strides=[1,1,1,1], padding = 'SAME')
print(L1)
bn1 = tf.layers.batch_normalization(L1, training=True)
at1 = tf.nn.relu(bn1)

pool1 =  tf.nn.max_pool(at1, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool1)

Tensor("Conv2D:0", shape=(?, 64, 64, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 32, 32, 32), dtype=float32)


In [7]:
W2 = tf.Variable(tf.random_normal([5, 5, 32, 64], stddev=0.01))

L2 = tf.nn.conv2d(pool1,W2, strides=[1,1,1,1], padding = 'SAME')
bn2 = tf.layers.batch_normalization(L2, training=True)
at2 = tf.nn.relu(bn2)

pool2 =  tf.nn.max_pool(at2, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool2)

Tensor("MaxPool_1:0", shape=(?, 16, 16, 64), dtype=float32)


In [8]:
W3 = tf.Variable(tf.random_normal([4, 4, 64, 128], stddev=0.01))

L3 = tf.nn.conv2d(pool2,W3, strides=[1,1,1,1], padding = 'SAME')
bn3 = tf.layers.batch_normalization(L3, training=True)
at3 = tf.nn.relu(bn3)

pool3 =  tf.nn.max_pool(at3, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool3)

Tensor("MaxPool_2:0", shape=(?, 8, 8, 128), dtype=float32)


In [9]:
W4 = tf.Variable(tf.random_normal([4, 4, 128, 256], stddev=0.01))

L4 = tf.nn.conv2d(pool3,W4, strides=[1,1,1,1], padding = 'SAME')
bn4 = tf.layers.batch_normalization(L4, training=True)
at4 = tf.nn.relu(bn4)

pool4 =  tf.nn.max_pool(at4, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool4)

Tensor("MaxPool_3:0", shape=(?, 4, 4, 256), dtype=float32)


In [10]:
W5 = tf.Variable(tf.random_normal([4*4*256, 1024], stddev=0.01))
fc1 = tf.reshape(pool4,[-1,4*4*256])
fc1 = tf.matmul(fc1, W5)
fc1 = tf.nn.relu(fc1)

fc1 = tf.nn.dropout(fc1,keep_prob)
print(fc1)

Tensor("dropout/mul:0", shape=(?, 1024), dtype=float32)


In [11]:
W6 = tf.Variable(tf.random_normal([1024,2350],stddev=0.01))
logit = tf.matmul(fc1, W6)
print(logit)

Tensor("MatMul_1:0", shape=(?, 2350), dtype=float32)


In [12]:
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit, labels=input_labels))
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)
print(loss)

Tensor("Mean:0", shape=(), dtype=float32)


In [13]:
label_max=tf.argmax(logit,1)
print(label_max)

correct_prediction = tf.equal(tf.argmax(logit,1),input_labels)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Tensor("ArgMax:0", shape=(?,), dtype=int64)


In [20]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
saver = tf.train.Saver()
batch_size=100

total_batch = int(numb_of_data / batch_size)
images, labels = read_and_decode(tf_record_filename_queue,batch_size)
print("Total Batch :%d" % (total_batch))
with tf.Session() as sess:
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    for epoch in range(100):
        for i in range(total_batch):
        #for i in range(batch_size):
            batch_images, batch_labels = sess.run([images,labels])
            sess.run([optimizer,loss],feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:0.5})
            train_acc=sess.run(accuracy, feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0})
            if(i%100==0):
                print('Epoch :%d | step %d, training accuracy %f' % (epoch,i, train_acc))
        save_path = saver.save(sess, "save\model2.ckpt")
    coord.request_stop()
    coord.join(threads)

Total Batch :6658
Epoch :0 | step 0, training accuracy 0.030000
Epoch :0 | step 100, training accuracy 0.020000
Epoch :0 | step 200, training accuracy 0.000000
Epoch :0 | step 300, training accuracy 0.090000
Epoch :0 | step 400, training accuracy 0.160000
Epoch :0 | step 500, training accuracy 0.200000
Epoch :0 | step 600, training accuracy 0.190000
Epoch :0 | step 700, training accuracy 0.420000
Epoch :0 | step 800, training accuracy 0.400000
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>, Input to reshape is a tensor with 1 values, but the requested shape has 4096
	 [[Node: Reshape_6 = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Cast_11, Reshape_6/shape)]]
	 [[Node: Reshape_6/_29 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_na

OutOfRangeError: RandomShuffleQueue '_41_shuffle_batch_5/random_shuffle_queue' is closed and has insufficient elements (requested 100, current size 11)
	 [[Node: shuffle_batch_5 = QueueDequeueManyV2[component_types=[DT_FLOAT, DT_INT64], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](shuffle_batch_5/random_shuffle_queue, shuffle_batch_5/n)]]

Caused by op 'shuffle_batch_5', defined at:
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\site-packages\ipykern

In [6]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())

batch_size=100

total_batch = int(numb_of_data / batch_size)
images, labels = read_and_decode(tf_record_filename_queue,batch_size)

with tf.Session() as sess:
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    for i in range(3):
        img, lab = sess.run([images,labels])
        print(lab)
    coord.request_stop()
    coord.join(threads)


[1256 1658 1333 1399  581 1487 1692 2177 2103   32 1418  347 2293 1178
  832 1215  323 1434  364 1989 1857  565 1386  154 1366 1040 2247 1346
 1982  822 1572  332 1456  568  560  648 1690  804 1582 1213  908 2304
  769 1631 1118 1256 1849 2104  427 1519  856 2209  446 1171  437 1787
 1652  474  161  347 2183  573 1358  582 1250 1529 1378 1692 1370 1180
 2110  938  308 1787  386 2154  861 1549   32  407  427 1368 2133 1594
 2214  199  344 1231 1104   86  282  938  186 1356 1213 1358  754 1779
  490  549]
[1572  861  249 1477  555 1126 2200  570  729  494 1563  128 1041  294
  582 1807 1217 1989  876  384 1720 1266 1652 1462 1554  987 1529 1405
 1013 1434 1780 1178 1782 1516  885 1825  699  985 2336 1258 1478  901
 1358 1550    4 1379    0 1880 2253 1430 1112 2258 2282  571  745  891
 1419  651  159 1055  697   42 1548  904 2028 2083 1574 2175  950 1436
  377 1247  647  329 1186 1859 1535  859 1462  448   36  729 1657 1652
 2009 2342 1647 1601 1937   42  107    4 1860 1747  856 1616 1890